In [1]:
"""Script permettant d'interroger une base de données SQL"""
################################################################################
# fichier  : td5_interrogation.py
# Auteur : RAKOTOSON Loic
################################################################################

"Script permettant d'interroger une base de données SQL"

In [2]:
################################################################################
# Importation de fonctions externes :
import json
import pandas as pd
import sqlite3 as db
from requests import get

# Definition locale de fonctions :

In [3]:
def getQuery(query, database = "base.sqlite"):
    """
    Intérroge et renvoie la réponse en DataFrame
    :query: str, requête SQL
    :database: str, base de données
    :return: DataFrame
    """
    cnx = db.connect(database)
    table = pd.read_sql_query(query, cnx)
    cnx.close()
    return table

In [4]:
def getMatrix(base, dico, metric):
    """
    Retourne la matrice de la métrique
    :base: base url
    :dico: API params
    :metric: distances|durations
    :return: DataFrame
    """
    
    def getJsonData(base, dico):
        with get(base, params = dico) as response:
            return response.json()
    
    reponse = getJsonData(base, dico)
    df = pd.DataFrame(
        reponse[metric],
        columns=[
            ",".join(map(str, d['location'])) for d in reponse['destinations']
        ],
        index=[",".join(map(str, d['location'])) for d in reponse['sources']]).T
    return df

In [5]:
################################################################################
# Corps principal du programme :

# Exercice 2: utilisation des données de la base
La fonction `getQuery( )` a été créée pour obtenir les réponses des requêtes SQL en tant que DataFrame.

En utilisant les données de la base et éventuellement d'autres ressources, trouver:

#### Question 1
Les titres, dates de début et de fin des événements de type «Exposition, musée» qui se sont déroulés (totalement ou partiellement) en février 2019.

In [6]:
query = """SELECT e_titre, e_datedebut, e_datefin
           FROM evenement
           WHERE "02" IN (strftime('%m', e_datedebut), strftime('%m', e_datefin))
           AND e_typeid = (SELECT te_id FROM typeEvenement WHERE te_nom = "Exposition, musée")
        """

getQuery(query)

,E_titre,E_dateDebut,E_dateFin
0,Exposition du PLU et PLUI,2018-02-14,2018-02-23
1,"« Qui a refroidi Lemaure ? », une BD interactive",2018-12-18,2019-02-02
2,Fermeture annuelle centre minier de Faymoreau,2018-11-06,2019-02-08
3,Ouverture gratuite du musée d’Art et d’Histoire,2019-02-03,2019-02-03
4,Deizioù 2019 : costumes de Bretagne,2019-02-05,2019-02-24
5,À l’écoute des oeuvres,2018-12-18,2019-02-05
6,Exposition : la forêt,2019-01-02,2019-02-23
7,Exposition de photographies,2019-01-11,2019-02-28
8,Couleurs du Tibet,2019-01-08,2019-02-16
9,Peintures animalières,2019-02-11,2019-02-28


#### Question 2 :
Les titres, dates de début et de fin, et la commune des événements affichés complets

In [7]:
query = """SELECT e_titre, e_datedebut, e_datefin, c_nom
           FROM evenement JOIN commune ON e_communeid = c_insee
           WHERE e_complet = 'True'
        """

getQuery(query)

,E_titre,E_dateDebut,E_dateFin,C_nom
0,Stage trapèze mini-volant découverte,2019-01-20,2019-01-20,Le Mans


#### Question 3 :
Les titres, dates de début et de fin, et la description des événements proposés par des organismes de type «Etablissement scolaire, universitaire» en Ille-et-Vilaine, avec le nom de l'organisme

In [8]:
query = """SELECT e_titre, e_datedebut, e_datefin, o_nom, e_description
           FROM evenement JOIN commune ON e_communeid = c_insee
                          JOIN organisme on e_organismeid = o_id
           WHERE o_type = 'Etablissement scolaire, universitaire'
           AND c_dep = 35
        """

getQuery(query)

,E_titre,E_dateDebut,E_dateFin,O_nom,E_description
0,Portes ouvertes de l école commmunale,2019-01-19,2019-01-19,Ecole communale,"Venez découvrir l équipe enseignante, les loca..."
1,Soutenance de thèse,2019-02-05,2019-02-05,"Agrocampus Ouest, centre de Rennes, coordinati...","Organisateur : Agrocampus ouest, centre de Ren..."


#### Question 4 :
Les titres, date de début, date de fin et département des événements non accessibles aux moins de 12 ans qui se déroulent en dehors de la Bretagne (c'est à dire hors des départements bretons: 22, 29, 35, 56)

In [9]:
query = """SELECT e_titre, e_datedebut, e_datefin, c_dep
           FROM evenement JOIN commune ON e_communeid = c_insee
           WHERE e_agemin >= 12
           AND c_dep NOT IN (22, 29, 35, 56)
        """

getQuery(query)

,E_titre,E_dateDebut,E_dateFin,C_dep
0,« L’homme au bras de mer »,2019-02-28,2019-02-28,61
1,Le versant sombre des bons sentiments,2019-01-06,2019-01-06,49
2,Initiation au modelage avec de l’argile,2018-12-15,2018-12-15,50
3,Rencontre littéraire avec Claude Bourgeyx,2019-01-12,2019-01-12,44
4,Self-défense pour femme,2019-01-19,2019-01-19,85
5,Fonds de développement de la vie associative,2019-01-30,2019-01-30,49
6,Confessions très intimes,2019-01-31,2019-02-02,13


#### Question 5 :
Les titres, date de début, date de fin et type (nom du type) des événements accessibles aux personnes ayant un handicap «Moteur», proposés en Ille-et-Vilaine par des associations, avec le nom de l’association.

In [10]:
query = """SELECT e_titre, e_datedebut, e_datefin, te_nom, o_nom
           FROM evenement JOIN commune ON e_communeid = c_insee
                          JOIN typeEvenement ON e_typeid = te_id
                          JOIN organisme ON e_organismeid = o_id
           WHERE e_accessibilite LIKE '%Moteur%'
           AND c_dep = 35
           AND o_type = 'Association'
        """

getQuery(query)

,E_titre,E_dateDebut,E_dateFin,TE_nom,O_nom
0,Repair café,2018-01-06,2018-01-06,"Atelier, activité de loisirs",Les Colocaterre
1,Entente OCM-JSB section basket,2018-02-17,2018-02-18,Sport,OCM basket
2,Randonnée pédestre,2018-03-25,2018-03-25,Randonnée,Association Bretagne solidarité Pérou Bolivie
3,Prévention et secours civiques de niveau 1,2019-03-16,2019-03-16,"Formation, vie scolaire, concours",Rennes Sauvetage
4,Prévention et secours civiques de niveau 1,2019-04-29,2019-04-29,"Formation, vie scolaire, concours",Rennes Sauvetage
5,Atelier cinéma,2018-10-06,2019-05-25,"Atelier, activité de loisirs",Compagnie Artefakt
6,Musique,2018-11-17,2018-11-17,Spectacle,La Montalbanaise
7,Réunion des alcooliques anonymes,2019-01-25,2019-01-25,"Réunion, assemblée",Alcooliques Anonymes
8,Réunion des alcooliques anonymes,2019-02-22,2019-02-22,"Réunion, assemblée",Alcooliques Anonymes
9,Entente Montauban-Boisgervilly-Médréac section...,2018-12-08,2018-12-08,Sport,OCM basket


#### Question 6 :
Les titres, dates de début et de fin, et les coordonnées GPS de tous les événements gratuits à Rennes dans le thème «Loisir» qui ont lieu après le 1er mars 2019 et qui se déroulent à moins de 45 minutes à pied du campus de Villejean, «Place du Recteur Henri Le Moal, 35000 Rennes».

In [11]:
query = """SELECT e_titre, e_datedebut, e_datefin, e_gps
           FROM evenement JOIN commune ON e_communeid = c_insee
                          JOIN typeEvenement ON e_typeid = te_id
           WHERE e_gratuit = 'oui'
           AND c_cp = 35000
           AND te_theme = 'Loisir'
           AND e_datedebut > DATE('2019-03-01')
        """

df = getQuery(query)

In [12]:
r2 = ['-1.701901,48.11981']
e_gps = [','.join(gps.split(', ')[::-1]) for gps in list(df.E_gps)]

base_url = "https://api.openrouteservice.org/matrix"
APIKEY = '5b3ce3597851110001cf62486cfda2a850de46e4a43253fa6adffd31'

params = {
    "api_key": APIKEY,
    "profile": "foot-walking",
    "locations": "|".join(r2 + e_gps),
    "sources": "0",
    "destinations": ",".join(map(str, range(1, len(r2 + e_gps)))),
    "metrics": "duration",
}

In [13]:
duration = getMatrix(base_url, params, "durations").reset_index(drop = True) / 60
duration.columns = ['duration']
df = pd.concat([df, duration], join = "inner", axis = 1)

df[df['duration'] < 45]

,E_titre,E_dateDebut,E_dateFin,E_gps,duration
0,J’irai graffer sur vos murs,2019-03-03,2019-03-03,"48.10511, -1.674704",42.046667
1,Sussistinako,2019-03-15,2019-03-15,"48.10511, -1.674704",42.046667
4,Malika Doray : rencontre-atelier,2019-03-13,2019-03-13,"48.10056, -1.709239",36.042833
5,Atelier numérique : L’offre en ligne,2019-03-22,2019-03-22,"48.11178, -1.68026",28.978167
6,Erwan Rouge et Lionel Monier,2019-05-31,2019-05-31,"48.11, -1.68",31.418833
7,Fest-noz avec Skolvan,2019-04-04,2019-04-04,"48.11934, -1.707685",6.264667
8,Théâtre cycles II et III : Lecture d’une œuvre...,2019-03-07,2019-03-07,"48.10929, -1.697211",18.221833
9,Orchestre symphonique,2019-03-07,2019-03-07,"48.11136, -1.678833",30.389333
10,Atelier de conversation en français,2019-03-19,2019-03-19,"48.10511, -1.674704",42.046667


#### Question 7 :
Les titres, dates de début et de fin et la commune des événements de type «Concert»  gratuits ou dont le tarif réduit est inférieur ou égal à 10 euros et situés à moins de 1h de voiture de votre domicile à Rennes (ou du campus de Villejean).

In [14]:
query = """SELECT e_titre, e_datedebut, e_datefin, c_nom, e_gps
           FROM evenement JOIN commune ON e_communeid = c_insee
                          JOIN typeEvenement ON e_typeid = te_id
           WHERE (e_gratuit = 'oui' OR e_tarifreduit <= 10)
           AND te_nom = 'Concert'
        """

df = getQuery(query)

In [15]:
e_gps = [','.join(gps.split(', ')[::-1]) for gps in list(df.E_gps)]
df = df.drop(['E_gps'], axis=1)

params = {
    "api_key": APIKEY,
    "profile": "driving-car",
    "locations": "|".join(r2 + e_gps),
    "sources": "0",
    "destinations": ",".join(map(str, range(1, len(r2 + e_gps)))),
    "metrics": "duration",
}

In [16]:
duration = getMatrix(base_url, params, "durations").reset_index(drop = True) / 3600
duration.columns = ['duration']
df = pd.concat([df, duration], join = "inner", axis = 1)

df[df['duration'] < 1]

,E_titre,E_dateDebut,E_dateFin,C_nom,duration
0,Les Concerts du jeudi,2017-12-28,2017-12-28,Avranches,0.886431
4,Humour Fellag Bled Runner,2018-02-16,2018-02-16,Lamballe-Armor,0.891031
7,Concert de Mirthe,2018-02-03,2018-02-03,Lamballe-Armor,0.859131
14,"« À travers chants » et « Amadé Oust""",2018-04-13,2018-04-13,Ploërmel,0.903089
16,Escapade du conservatoire,2018-05-18,2018-05-18,Rives-du-Couesnon,0.487636
22,"Éclats d’opéras, par l’Ensemble vocal et instr...",2018-05-27,2018-05-27,Avranches,0.875714
25,Les Concerts du jeudi,2018-05-31,2018-05-31,Avranches,0.886431
26,"Ateliers théâtre, Comédie « Surprise » de Cath...",2018-06-20,2018-06-20,Avranches,0.870889
45,Les Concerts du jeudi,2018-09-27,2018-09-27,Avranches,0.886431
47,The Horny Tonky,2018-11-02,2018-11-02,Lamballe-Armor,0.891031
